**Install Packages**

In [2]:
import os, sys
# import seaborn as sns
import pandas as pd
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sqlalchemy as sa
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import timedelta, date
%matplotlib inline

**Initalise Database Connection and insert SQL Query**

In [3]:
def query_arc_alchemy(sql):
    # ODBC connection string
    _url ='mssql+pyodbc:///?odbc_connect=DATABASE%3DARCTIMESERIES%3BDRIVER%3D%7BSQL+Server+Native+Client+11.0%7D%3BSERVER%3DARCSQL%3BTrace%3D0%3BTrusted_Connection%3DYes'
 

    engine =sa.create_engine(_url, echo=False, convert_unicode=True)
    with engine.connect() as connection:
        result =connection.execute(sql)
        data =result.fetchall()
        column_names =result.keys()
        dataframe = pd.DataFrame.from_records(data, columns=column_names)

        return dataframe

query_str_1 = f"""
Select c.CurveName, c.EffectiveDate,cast(format(c.PointDate, 'MM-01-yyyy') as Date) as PointMonth, avg(c.PointValue) as value
FROM [ArcTimeSeries].[arc_ts_data].[PriceCurveData] c 
where c.PointDate>c.EffectiveDate and c.EffectiveDate>'01-01-2019'
and DATEDIFF(m,c.EffectiveDate,c.PointDate)>=1 and DATEDIFF(m,c.EffectiveDate,c.PointDate)<24
and c.EffectiveDate not between '2019-09-30' and '2019-10-01'
and c.CurveName in (Select Distinct(c.CurveName)
FROM [ArcTimeSeries].[arc_ts_data].[PriceCurveData] c 
where c.CurveName like '%LNG_Country_Supply_Forecast_Historics%'
or c.CurveName like '%LNG_Country_Total_Demand_Forecast_Historics%')
group by c.CurveName, c.EffectiveDate,cast(format(c.PointDate, 'MM-01-yyyy') as Date)
order by 1,2,3
"""


query_str_2 = f"""
Select e.CurveName, e.EffectiveDate, e.PointMonth,  e.value, e.rank_no
FROM
(Select dense_rank() over (partition by d.CurveName,d.PointMonth order by d.EffectiveDate DESC)
    as rank_no,
d.CurveName,
d.EffectiveDate, 
d.PointMonth, 
d.value

From (Select c.CurveName, c.EffectiveDate,cast(format(c.PointDate, 'MM-01-yyyy') as Date) as PointMonth, avg(c.PointValue) as value
FROM [ArcTimeSeries].[arc_ts_data].[PriceCurveData] c 
where c.PointDate<c.EffectiveDate and c.EffectiveDate>'01-01-2019'
and DATEDIFF(m,c.EffectiveDate,c.PointDate)<=-1
and c.EffectiveDate not between '2019-09-30' and '2019-10-01'
and c.PointDate>='01-01-2019'
and c.CurveName in (Select Distinct(c.CurveName)
FROM [ArcTimeSeries].[arc_ts_data].[PriceCurveData] c 
where c.CurveName like '%LNG_Country_Supply_Forecast_Historics%'
or c.CurveName like '%LNG_Country_Total_Demand_Forecast_Historics%')
group by c.CurveName, c.EffectiveDate,cast(format(c.PointDate, 'MM-01-yyyy') as Date)) d) e

where e.rank_no = 1

order by 1,3,2
"""

**Define Function which allows you to pull the last forecast per row and visualise Dataframes**

In [4]:
def f(row):
    return [t for t in row.values.tolist() if t is not np.NaN][-1]

In [5]:
from IPython import display

In [6]:
def display_full_df(df):
    if isinstance(df, pd.Series):
        df = df.to_frame()
    pd.set_option('display.max_rows', pd.DataFrame(df).shape[0])
    pd.set_option('display.max_columns', pd.DataFrame(df).shape[1])
    pd.set_option('display.max_colwidth', 1000)
    display.display(pd.DataFrame(df))

    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.max_colwidth')

**Pull Queries into Dataframe**

In [7]:
forecast = query_arc_alchemy(query_str_1)

In [8]:
Actual = query_arc_alchemy(query_str_2)

**Create Unique list of curvenames**

In [9]:
unique_curvenames = forecast['CurveName'].unique().tolist()

**Re-name curvenames**

In [10]:
new_name = []

for idx,t in enumerate(unique_curvenames):
    
    if unique_curvenames[idx][0:38] == 'LNG_Country_Supply_Forecast_Historics_':
        new_name.append('Supply_' + unique_curvenames[idx][38:])
    else:
        new_name.append('Demand_' + unique_curvenames[idx][44:])

**Initialise forecast_dict and store history and forecast**

In [12]:
forecast_dict = {}

for idx,t in enumerate(unique_curvenames[:3]):
    
    #re-organise forecasts into a pivottable with min,max,last front month forecast
    filter_curve = forecast['CurveName']==t
    
    DF = forecast[filter_curve]
    
    DF_pivot = pd.pivot_table(DF,index=pd.DatetimeIndex(DF['PointMonth']),columns='EffectiveDate',values='value',aggfunc= np.sum)
    
    DF_pivot['last Front Month Forecast'] = DF_pivot.apply(lambda row: f(row)  , axis=1)
    DF_pivot['Min'] = DF_pivot.min(axis=1,skipna=True)
    DF_pivot['Max'] = DF_pivot.iloc[:,0:-3].max(axis=1,skipna=True)
    DF_pivot['Range'] = DF_pivot['Max'] - DF_pivot['Min']
    
    #organise actual imports/exports and join to DF_pivot to get a comprehensive dataset of: min,max,last front month forecast and actual
    
    actual_curve = Actual['CurveName'] == t
    actual_DF = Actual[actual_curve]
    
    actual_pivot = pd.pivot_table(actual_DF,index=pd.DatetimeIndex(actual_DF['PointMonth']),values='value',aggfunc= np.sum)
    
    actual_pivot.columns = ['Actual']
    
    
    #join two dataframes
    union_DF = DF_pivot.join(actual_pivot,how='left',on='PointMonth')
    
    #store in dictionary using the new name as the key 
    forecast_dict[new_name[idx]] = union_DF[['Min','Max','Range','last Front Month Forecast','Actual']]
    
    

    
    

In [14]:
forecast_dict['Supply_Algeria']

Min       Max     Range last Front Month Forecast  \
PointMonth                                                           
2019-10-01  1.011989  1.011989  0.000000            1.011989226545   
2019-11-01  0.983568  0.983568  0.000000            0.983567541441   
2019-12-01  0.967782  1.050448  0.082665            1.050447508029   
2020-01-01  0.890422  0.994940  0.104518            0.890422005036   
2020-02-01  0.913527  0.971631  0.058104            0.971455581784   
2020-03-01  1.103561  1.199846  0.096284            1.199845535219   
2020-04-01  0.903036  0.991370  0.088334            0.991304571778   
2020-05-01  1.012171  1.089454  0.077283            1.028436313154   
2020-06-01  0.959398  1.059975  0.100577            0.975588341444   
2020-07-01  0.772223  0.913293  0.141071            0.776564393373   
2020-08-01  0.954517  0.975264  0.020748            0.958813222582   
2020-09-01  0.953698  0.967725  0.014027            0.967568411808   
2020-10-01  1.010152  1.064562  0.054409            1.010152259270   
2020-11-01  0.917625  1.091552  0.173927            0.917625062754   
2020-12-01  0.966972  1.084295  0.117323            1.009999492022   
2021-01-01  0.865438  1.040679  0.175240            0.865438313008   
2021-02-01  0.876693  0.953350  0.076657            0.880146748251   
2021-03-01  1.058734  1.186605  0.127871            1.101622924606   
2021-04-01  0.878211  1.004368  0.126157            0.878211329532   
2021-05-01  0.916330  1.120645  0.204315            0.916330152910   
2021-06-01  0.903342  1.082937  0.179596            0.903341666319   
2021-07-01  0.684836  0.872930  0.188094            0.795766543891   
2021-08-01  0.883732  0.958813  0.075081            0.883732378795   
2021-09-01  0.862064  0.967568  0.105504            0.862063966473   
2021-10-01  0.970478  1.028289  0.057811            0.970478210019   
2021-11-01  0.956251  1.059212  0.102961            0.956251137416   
2021-12-01  0.944599  1.116134  0.171535            0.944598881923   
2022-01-01  0.812433  0.938122  0.125690            0.861272018937   
2022-02-01  0.875910  1.028383  0.152473            0.875909646515   
2022-03-01  1.096320  1.275450  0.179130            1.096319617612   
2022-04-01  0.873984  1.065221  0.191238            0.873983545067   
2022-05-01  0.911919  1.038560  0.126641            0.911918861168   
2022-06-01  0.898993  0.968631  0.069638            0.898992902263   
2022-07-01  0.689208  0.791936  0.102728            0.791935655677   
2022-08-01  0.879478  0.964829  0.085351            0.879478015526   
2022-09-01  0.857914  0.957903  0.099989            0.857913916795   
2022-10-01  0.965806  1.006541  0.040735            0.965806244898   
2022-11-01  0.951648  0.951648  0.000000            0.951647662640   
2022-12-01  0.940052  0.940052  0.000000            0.940051502102   

                    Actual  
PointMonth                  
2019-10-01  1.097115687250  
2019-11-01  1.123364047046  
2019-12-01  1.006733098401  
2020-01-01  0.815528170397  
2020-02-01  0.833306629537  
2020-03-01  0.963251044782  
2020-04-01  0.963914001078  
2020-05-01  1.068537923638  
2020-06-01  1.022613709292  
2020-07-01  0.840908437830  
2020-08-01  0.846653153384  
2020-09-01  0.692019050181  
2020-10-01  0.869041060705  
2020-11-01  0.921119809900  
2020-12-01  0.782459602422  
2021-01-01             NaN  
2021-02-01             NaN  
2021-03-01             NaN  
2021-04-01             NaN  
2021-05-01             NaN  
2021-06-01             NaN  
2021-07-01             NaN  
2021-08-01             NaN  
2021-09-01             NaN  
2021-10-01             NaN  
2021-11-01             NaN  
2021-12-01             NaN  
2022-01-01             NaN  
2022-02-01             NaN  
2022-03-01             NaN  
2022-04-01             NaN  
2022-05-01             NaN  
2022-06-01             NaN  
2022-07-01             NaN  
2022-08-01             NaN  
2022-09-01             NaN  
2022-10-01             NaN  
2022-11-01     

**Plot Output to check**

In [ ]:
plt.plot(forecast_dict['Demand_Thailand'][['Min','Max','last Front Month Forecast','Actual']])

## Save to excel

In [16]:
file_name = 'my_report.xlsx'
with pd.ExcelWriter(file_name, engine='xlsxwriter', date_format='yyyy-mm-dd') as writer:
    for key in forecast_dict.keys():
        forecast_dict[key].to_excel(writer, sheet_name=key)
    